# Подготовка рекомендательной системы для покупки курсов в онлайн-школе MasterMind

**ЦЕЛЬ**<br>
Подготовить основу рекомендательной системы, благодаря которой можно будет предлагать клиентам интересные им курсы и тем самым повышать средний чек.

**ЗАДАЧИ**<br>
Оформление таблицы, в которой каждому курсу будет соответствовать по два рекомендованных к покупке курса.

**КОНКРЕТНЫЕ ШАГИ (ФОРМАЛИЗОВАННЫЕ ЗАДАЧИ)**<br>
- Проанализировать базу данных о продажах за 2017-2018 гг. с помощью SQL
- Выгрузить из БД необходимые данные
- Обработать данные в Python
- Получить рекомендательную таблицу

In [83]:
# импортируем все необходимые библиотеки и модули
import pandas as pd
import numpy as np
from itertools import combinations
from collections import Counter
import psycopg2

Производим работу с базой данных с помощью SQL - запроса. Формируем таблицу, содержащую перечень всех пользователей, купивших более 1 курса, а также id купеленных ими курсов

In [84]:
# запрос к БД содержащий SQL код
try: 
    connection = psycopg2.connect( #параметры подключения к БД
        dbname='skillfactory', 
        user='skillfactory',
        host='84.201.134.129',
        password='cCkxxLVrDE8EbvjueeMedPKt',
        port=5432,
    )
    cursor = connection.cursor()
    #SQL код, содержащий CTE (выдает перечень клиентов, которые покупали больше 1 курса, а также количество купелнных ими курсов)
    # и основной запрос (выдает перечень клиентов, которые покупали больше 1 курса, а также id купленных ими курсов)
    postgres_select_query = """ WITH interested_clients AS
(
SELECT user_id,
COUNT (DISTINCT resource_id) AS quantity
FROM final.carts c
JOIN final.cart_items i ON i.cart_id = c.id
WHERE state = 'successful'
AND resource_type = 'Course'
GROUP BY 1
HAVING COUNT (DISTINCT resource_id) > 1
)
SELECT DISTINCT
c.user_id, 
resource_id
FROM final.carts c
JOIN final.cart_items i ON i.cart_id = c.id
JOIN interested_clients ic ON ic.user_id = c.user_id
WHERE state = 'successful' AND resource_type = 'Course'
ORDER BY 1 """
    cursor.execute(postgres_select_query)
    courses_df=cursor.fetchall()
    connection.commit()
   
except (Exception, psycopg2.Error) as error:
    if connection:
        print("Error", error)
finally:
    if connection:
        cursor.close()
        connection.close()

In [85]:
# преобразуем в DataFrame полученную благодаря SQL-запросу таблицу
df = pd.DataFrame(courses_df)
# переименовываем столбцы 
df = df.rename(columns={0:'user_id',1:'resource_id'})
# группируем id курсов в списки по id клиентов
combined_courses = df.groupby('user_id')['resource_id'].apply(list)
#подсчитываем количество повторяющихся сочетаний купленных курсов
combined_courses.value_counts()

[551, 566]                                     431
[566, 794]                                     146
[490, 566]                                     102
[515, 551]                                      91
[490, 551]                                      87
                                              ... 
[519, 523, 562]                                  1
[366, 523, 566, 809, 1115]                       1
[509, 516, 553, 568, 745, 1099, 1103, 1125]      1
[551, 1101]                                      1
[756, 794, 1185]                                 1
Name: resource_id, Length: 5483, dtype: int64

Разбиваем все курсы по парам, ранжируем их для каждого курса и считаем, сколько различных пар курсов встречается вместе в покупках клиентов

In [86]:
combined_courses_df = combined_courses.to_frame()
couples_of_courses = combined_courses_df['resource_id'].apply(lambda x:list(combinations(np.unique(x),2)))
couples_of_courses_df = couples_of_courses.to_frame()
couples_of_courses_list = couples_of_courses_df['resource_id'].tolist()
couples = set()
for item in couples_of_courses_list:
    for i in item:
        couples.add(i)
len(couples)

3989

Подготовим список пар курсов для всех пользоватлей (содержит повторы), после чего найдем наиболее часто встречающиеся пары

In [87]:
list_for_count = list()
for item in couples_of_courses_list:
    for i in item:
        list_for_count.append(i)
        
cnt = Counter(list_for_count)

cnt.most_common(1)

[((551, 566), 797)]

С помощью SQL-запроса создаем таблицу рейтинга курсов, в которой первая колонка содержит id курса, а вторая - количество пользователей, купивших курс

In [88]:
# запрос к БД содержащий SQL код
try: 
    connection = psycopg2.connect( #параметры подключения к БД
        dbname='skillfactory', 
        user='skillfactory',
        host='84.201.134.129',
        password='cCkxxLVrDE8EbvjueeMedPKt',
        port=5432,
    )
    cursor = connection.cursor()
    #SQL код
    postgres_select_query = """ SELECT DISTINCT
resource_id,
count(user_id) as rating
FROM final.carts c
JOIN final.cart_items i ON i.cart_id = c.id
WHERE state = 'successful' AND resource_type = 'Course'
GROUP BY 1 
ORDER BY 2 desc """
    cursor.execute(postgres_select_query)
    rating_courses_df=cursor.fetchall()
    connection.commit()
   
except (Exception, psycopg2.Error) as error:
    if connection:
        print("Error", error)
finally:
    if connection:
        cursor.close()
        connection.close()

In [89]:
rating_courses_df = pd.DataFrame(rating_courses_df) # преобразуем в DataFrame
rating_courses_df = rating_courses_df.rename(columns={0:'resource_id',1:'rating'}) #переименовываем столбцы 
rating_courses_df.head()

,resource_id,rating
0,551,9570
1,566,6222
2,490,2712
3,794,2525
4,515,2049


Создаем список уникальных курсов: в список уникальных курсов включены все курсы из представленных данных, в том числе те, которые не покупались. Поэтому список уникальных курсов больше количества курсов, представленных в рейтинге курсов

In [90]:
# запрос к БД содержащий SQL код
try: 
    connection = psycopg2.connect( #параметры подключения к БД
        dbname='skillfactory', 
        user='skillfactory',
        host='84.201.134.129',
        password='cCkxxLVrDE8EbvjueeMedPKt',
        port=5432,
    )
    cursor = connection.cursor()
    #SQL код
    postgres_select_query = """ SELECT 
    resource_id
    FROM  final.cart_items
    where resource_type = 'Course' """
    cursor.execute(postgres_select_query)
    cart_items_df =cursor.fetchall()
    connection.commit()
   
except (Exception, psycopg2.Error) as error:
    if connection:
        print("Error", error)
finally:
    if connection:
        cursor.close()
        connection.close()

In [91]:
cart_items_df = pd.DataFrame(cart_items_df) # преобразуем в DataFrame
cart_items_df = cart_items_df.rename(columns={0:'resource_id'}) #переименовываем столбцы 
courses_list = cart_items_df['resource_id'].unique()

Создаем датафрейм с рейтингом пар курсов

In [92]:
couples_list = cnt.most_common()
couples_list_df = pd.DataFrame(data=couples_list, columns = ['couples_courses', 'rating'])
couples_list_df.head()

,couples_courses,rating
0,"(551, 566)",797
1,"(515, 551)",417
2,"(489, 551)",311
3,"(523, 551)",304
4,"(566, 794)",290


Для того, чтобы установить минимальную границу при работе с популярностью пар курсов, найдем среднее, медиану (второй квартиль), а также верхний и нижний квартили

In [93]:
couples_list_df['rating'].mean()

10.031837553271497

In [94]:
couples_list_df.rating.quantile([0.25,0.5,0.75])

0.25    1.0
0.50    3.0
0.75    9.0
Name: rating, dtype: float64

Таким образом 75% сочетаний пар курсов покупались менее 9 раз

В связи с этим полагаем, что целесообразно установить нижнюю границу равную 9:
- если сочетание пар курсов встречалось менее 9 раз, тогда в качестве рекомендации следует брать первые два самые покупаемые курса ( датафрейм rating_courses_df: recommend_1 = 551, recommend_2 =566)

In [95]:
#делаем словарь пар курсов, в котором ключ - это пара курсов, а значение - это частота покупки такой пары
couples_dict = dict(cnt.most_common())

Прописываем функцию для поиска рекомендаций для курсов

In [96]:
def recomm(id):
    recomm_list = []
    for i in couples_dict.keys():
        if i[0] == id:
            recomm_list.append((i, couples_dict[i]))
        elif i[1] == id:
            recomm_list.append((i, couples_dict[i]))
    recomm_list_sort = sorted(recomm_list, key=lambda x:x[1], reverse=True)
    return recomm_list_sort[:2]  

Осуществляем итерацию по всему списку курсов, чтобы заполнить таблицу с рекомендациями.
При этому учитываем следующее:
- мы обусловились, что частота покупки пар кусров для рекомендации не может быть меньше 9;
- в случае, если частота покупки будет меньше 9 для второй рекомендации, то в качестве второй рекомендации фиксируем наиболее часто покупаемый курс (id 551);
- в случае, если частота покупки для обеих рекомендация будет меньше 9, то в качестве первой рекомендации фиксируем наиболее часто покупаемый курс (id 551), а в качестве второй рекомендации - второй наиболее часто покупаемый курс (id 566)

In [97]:
recommendation_df = pd.DataFrame(columns=['cours_id', 'first_recommendation', 'second_recommendation'])
recommend_1 = 551
recommend_2 =566
for i in courses_list:
    if len(recomm(i))==2:
        if recomm(i)[0][1]<9 and recomm(i)[1][1]<9:
            recommendation_df.loc[i] = [i, recommend_1, recommend_2]
        elif recomm(i)[0][1]>=9 and recomm(i)[1][1]<9:
            recommendation_df.loc[i] = [i, recomm(i)[0][0][1], recommend_1]
        elif recomm(i)[0][1]>=9 and recomm(i)[1][1]>=9:
            if i!=recomm(i)[1][0][1]:
                recommendation_df.loc[i] = [i, recomm(i)[0][0][1], recomm(i)[1][0][1]]
            elif i== recomm(i)[1][0][1]:
                recommendation_df.loc[i] = [i, recomm(i)[0][0][1], recomm(i)[1][0][0]]
    elif len(recomm(i)) == 1:
        if recomm(i)[0][1]<9:
            recommendation_df.loc[i] = [i, recommend_1, recommend_2]
        elif recomm(i)[0][1]>=9:
            recommendation_df.loc[i] = [i, recomm(i)[0][0][1], recommend_1]
    elif len(recomm(i)) == 0:
        recommendation_df.loc[i] = [i, recommend_1, recommend_2]  

Сортируем итоговую таблицу и делаем сброс индексов

In [98]:
recommendation_df = recommendation_df.sort_values('cours_id', ignore_index=True)
recommendation_df.head()

,cours_id,first_recommendation,second_recommendation
0,356,571,357
1,357,571,356
2,358,570,752
3,359,570,358
4,360,745,516


Проверяем факт того, что все курсы вошли в созданную таблицу

In [99]:
len(courses_list) == recommendation_df['cours_id'].count()

True


Полученная таблица готова для внедрения на сайт онлайн-школы. Для определения эффективности необходимо запустить А/В тесты.